In [ ]:
!pip install -U duckduckgo_search
!pip install anthropic

In [161]:
from duckduckgo_search import ddg

keywords = 'Wo steht der Dax?'
results = ddg(keywords, region='de', safesearch='On', time='m', max_results=3)

prompt=f"""Beantworten Sie die Frage am Ende des Textes anhand der folgenden Informationen:
### Eingabe
{{context}}
Frage: {{question}}
"""

formatted_prompt=prompt.format(context="\n".join([r["body"] for r in results]), question=keywords)
print(formatted_prompt)

Beantworten Sie die Frage am Ende des Textes anhand der folgenden Informationen:
### Eingabe
Der DAX ist ein sogenannter Performance-Index und misst dabei den Wertzuwachs von Kapitalanlagen. Bei der Berechnung des Performance-Index werden Kapitalveränderungen und Ausschüttungen von...
DAX (Kursindex) heute aktuell DAX (Kursindex)-Kurs in Realtime, DAX (Kursindex)-Chart, DAX (Kursindex) aktuell & live, Empfehlungen, gratis Analysen & Prognosen - boerse.de: Europas erstes ...
Der Dax liegt zum Handelsschluss etwa 0,4 Prozent höher und notiert bei 15.655 Punkten. Mit dem heutigen Tageshoch in Form von 15.726 Punkten fehlten dem deutsche Leitindex nur elf Zähler, um...
Frage: Wo steht der Dax?



In [162]:
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from duckduckgo_search import ddg

class CustomSearchTool(BaseTool):
    name = "Search"
    description = "A search engine. Useful for when you need to answer questions about current events. Input should be a search query."

    def _run(self, query: str) -> str:
        """Use the tool."""
        return ddg(query, region='en-Us', safesearch='On', time='y', max_results=3)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Duckduck go does not support async")

In [50]:
import importlib.util
import logging
from typing import Any, List, Mapping, Optional

from pydantic import Extra

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


class HuggingFacePipeline(LLM):
    pipeline: Any  #: :meta private:
    generation_kwargs: dict

    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.pipeline(prompt, **self.generation_kwargs)
        if self.pipeline.task == "text-generation":
            # Text generation return includes the starter text.
            text = response[0]["generated_text"][len(prompt) :]
        elif self.pipeline.task == "text2text-generation":
            text = response[0]["generated_text"]
        else:
            raise ValueError(
                f"Got invalid task {self.pipeline.task}, "
            )
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [3]:

    
model_id = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /opt/conda/envs/pytorch/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/envs/pytorch/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_4269/1540983265.py:20 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4269/1540983265.py'                         │
│                                                                                                  │
│ /home/ubuntu/langchain-tests/pydantic/main.py:341 in pydantic.main.BaseModel.__init__            │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/home/ubuntu/langchain-tests/pydantic/main.py'             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for HuggingFacePipeline
generate_kwargs
  extra fields not permitted (type=value_error.extra)

In [51]:

generate_kwargs = dict(
    top_p=1,
    top_k=0,
    temperature=0.001,
    do_sample=True,
    max_new_tokens=2048,
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)


llm = HuggingFacePipeline(pipeline=pipe,generation_kwargs=generate_kwargs)

In [53]:
text = "<|prompter|>Who is the president?<|endoftext|><|assistant|>"
print(llm(text))

As of February 2023, Joe Biden is the 46th and current President of the United States.


In [165]:
from langchain.llms import Anthropic
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.tools import AIPluginTool
from langchain.utilities import GoogleSerperAPIWrapper


# TODO: doesn'T work
PREFIX = """<|prompter|>Answer the following questions as best you can. You have access to the following tools:"""
SUFFIX = """Begin!
Question: {input}
\n<|endoftext|><|assistant|>
Thought:{agent_scratchpad}<|endoftext|>"""

# tool = AIPluginTool.from_plugin_url("https://www.klarna.com/.well-known/ai-plugin.json")
tool = CustomSearchTool()
tools = load_tools(["wikipedia"] )
tools += [tool]

agent_kwargs = {"prefix": PREFIX, "suffix": SUFFIX}
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, agent_kwargs=agent_kwargs
)
print(agent_chain.agent.llm_chain.prompt.template)

<|prompter|>Answer the following questions as best you can. You have access to the following tools:

Wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, historical events, or other subjects. Input should be a search query.
Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Wikipedia, Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}

<|endoftext|><|assistant|>
Thought:{agent_scratchpad}<|endoftext|>


In [ ]:
agent_chain.run("What is the capital of Germany?")

In [150]:
tmp = """<|prompter|>Question: Who lived longer, Muhammad Ali or Alan Turing?<|endoftext|><|assistant|>
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

<|prompter|>Question: When was the founder of craigslist born?<|endoftext|><|assistant|>
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

<|prompter|>Question: Who was the maternal grandfather of George Washington?<|endoftext|><|assistant|>
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

<|prompter|>Question: Are both the directors of Jaws and Casino Royale from the same country?<|endoftext|><|assistant|>
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate answer: New Zealand.
So the final answer is: No

<|prompter|>Question: {input}<|endoftext|><|assistant|>
Are followup questions needed here:{agent_scratchpad}"""

In [187]:
import os
os.environ["SERPER_API_KEY"] = "0605930b7bcc70497b8d0a6a37dad17ec000f837"

In [188]:
from langchain.llms import Anthropic
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.tools import AIPluginTool
from langchain.utilities import GoogleSerperAPIWrapper


tool_names = [""]
# tool = load_tools(["wikipedia"])[0]
# tool = CustomSearchTool()

tool = load_tools(["google-serper"])[0]
tools = [
    Tool(
        name="Intermediate Answer",
        func=tool.run,
        description=tool.description,
    )
]

agent_chain = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
agent_chain.agent.llm_chain.prompt.template = tmp


In [193]:
agent_chain.run("Who was the first German female canchellor?")



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who was the first German female canchellor?
Intermediate answer: Merkel was the first female chancellor of Germany. During her tenure as Chancellor, Merkel was frequently referred to as the de facto leader of the European Union (EU), the most powerful woman in the world, and since 2016 the leader of the free world.Follow up: What was Merkel's political career prior to becoming Chancellor?
Intermediate answer: Scientific career She obtained a doctorate in quantum chemistry in 1986 and worked as a research scientist until 1989. Merkel entered politics in the wake of the Revolutions of 1989, briefly serving as deputy spokeswoman for the first democratically elected Government of East Germany led by Lothar de Maizière.Follow up: What was Merkel's political career after becoming Chancellor?
Intermediate answer: A member of the Christian Democratic Union (CDU), she previously served as Leader of the Opposition from 2002 to 2005 and as 

'Merkel'

In [179]:
llm("<|prompter|>Who was the first German chancellor?<|endoftext|><|assistant|>")

'The first German chancellor was Otto von Bismarck, who served from 1871 to 1890.'